In [19]:
import pandas as pd
import sys
sys.path.append('../')
sys.path.append('../potzy/')

#LOG IN INTO SIAMCHART
import potzy.metadata
import potzy.helpers
import potzy.morningstarreader as ms

%load_ext autoreload
%autoreload

all_tickers = potzy.metadata.get_all_thai_tickers()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
df=pd.DataFrame()
bad_tickers=set()
for ticker in all_tickers:
#for ticker in ['CPALL','HMPRO']:
    try:
        dfs = ms.load_key_ratio_thai(ticker).reset_index()
        dfs['ticker']=ticker
        
        tmp=set([int(x.split('-')[-1]) for x in dfs.columns if '-' in x])
        if len(tmp) > 1:
            print(ticker,": Multiple end-of-year months: Weird.")
            print(ticker,": Checking columns ... ")
            print(ticker,": ",dfs.columns)
        dfs['endofyear_month']= next(iter(tmp))
        
        #replace columns
        dfs.columns = [x.split('-')[0] for x in dfs.columns]
        
        #append to 
        df = df.append(dfs,ignore_index=True)
        
    except:
        bad_tickers.add(ticker)

APEX : Multiple end-of-year months: Weird.
APEX : Checking columns ... 
APEX :  Index(['item', '2006-06', '2007-06', '2009-12', '2010-12', '2011-12',
       '2012-12', '2013-12', '2014-12', '2015-12', '2016-12', 'TTM', 'ticker'],
      dtype='object')
CNS : Multiple end-of-year months: Weird.
CNS : Checking columns ... 
CNS :  Index(['item', '2007-08', '2008-08', '2009-08', '2010-08', '2011-08',
       '2012-12', '2013-12', '2014-12', '2015-12', '2016-12', 'TTM', 'ticker'],
      dtype='object')
EASTW : Multiple end-of-year months: Weird.
EASTW : Checking columns ... 
EASTW :  Index(['item', '2007-09', '2008-09', '2009-12', '2010-12', '2011-12',
       '2012-12', '2013-12', '2014-12', '2015-12', '2016-12', 'TTM', 'ticker'],
      dtype='object')
KBS : Multiple end-of-year months: Weird.
KBS : Checking columns ... 
KBS :  Index(['item', '2008-09', '2009-09', '2010-09', '2011-09', '2011-12',
       '2012-12', '2013-12', '2014-12', '2015-12', '2016-12', 'TTM', 'ticker'],
      dtype='obje

In [21]:
#count numbaer of tickers
len(bad_tickers),len(df.ticker.unique())

(62, 684)

In [22]:
output_file = "../data/morningstar/ks_thai_all.csv"
potzy.helpers.ensure_dir(output_file)
df.to_csv(output_file)